In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import csv
from datetime import datetime

/home/lucia/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


* Mood vector Design 1: for each day, if negative_count > positive_count or negative_count > neutral_count or neative_count > mix_count, assign negative to that day; if positive_count > negative_count or positive_count > neutral_count or positive_count > mix_count, assign positive to that day; if mix_count >= positive_count or mix_count >= negative_count or negative_count == positive_count, assign mix to that day; if neutral_count >= positive_count or neutral_count >= negative_count or neutral_count > mix_count, assign neutral to that day.

In [4]:
#path = '/Users/lucia/phd_work/cognitive_distortion/'
path = '/home/lucia/phd_work/mypersonality_data/cognitive_distortion/'
time = pd.read_csv(path + '/data/important_data/twoM_newLabels.csv')
ids = pd.read_csv(path + '/data/important_data/Id80PerRetained.csv')
time  = time[['text','id','negative_ny','userid','time','time_diff']]
time.shape

(4099, 6)

In [5]:
ids2 = ids[['userid']]
time = pd.merge(ids2, time, how = 'left', on = 'userid' )
time.shape

(4086, 6)

In [6]:
#select date
time['time'] = time['time'].apply(lambda x: x.split()[0])
time[0:5]

,userid,text,id,negative_ny,time,time_diff
0,009d96a823b6f6c085c092fb177491f6,Happy mothers day to all the privileged beauti...,950,2,08/05/2011,4
1,009d96a823b6f6c085c092fb177491f6,"In honor of Mother's Day, we are trying to see...",948,4,05/05/2011,7
2,009d96a823b6f6c085c092fb177491f6,"What would your liqour do? If it could, it wou...",949,2,05/05/2011,7
3,009d96a823b6f6c085c092fb177491f6,War is about who's left not who's right.... be...,946,2,04/05/2011,8
4,009d96a823b6f6c085c092fb177491f6,Loving the fact my babie made me breakfast mmm...,942,2,01/05/2011,11


In [7]:
#convert to time series
# time['time'] = time['time'].apply(lambda x: parser.parse(x))
# time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
time['time'][1:10]

1   2011-05-05
2   2011-05-05
3   2011-05-04
4   2011-05-01
5   2011-04-28
6   2011-04-25
7   2011-04-20
8   2011-04-20
9   2011-04-19
Name: time, dtype: datetime64[ns]

In [33]:
#sort values according to userid, time and valence
time = time.sort_values(by=['userid','time_diff','negative_ny'],  ascending=False)

#time.to_csv(path + '/data/important_data/time.csv')

In [60]:
def getTransitions(ValenceObject):
    emptyTran = 0
    negaTran = 0
    posiTran = 0
    mixTran = 0
    neuTran = 0
    PosAndNeg = 0
    MixAndPos = 0
    MixAndNeg = 0
    MixAndNeu = 0    
    NeuAndPos = 0
    NeuAndNeg = 0
    EmptyAndPos = 0
    EmptyAndNeg = 0
    EmptyAndMix = 0
    EmptyAndNeu = 0
    preValence = 0
    for valence in ValenceObject:
    #these are self transition states
        if valence == 0 and preValence == 0:
            emptyTran = emptyTran + 1
        elif valence == 1 and preValence == 1:
            negaTran = negaTran + 1
        elif valence == 2 and preValence == 2:
            posiTran = posiTran + 1
        elif valence == 3 and preValence == 3:
            mixTran = mixTran + 1
        elif valence == 4 and preValence == 4:
            neuTran = neuTran + 1
    #positive and negative transition:
        if (valence == 1 and preValence == 2) or (valence == 2 and preValence == 1) :
            PosAndNeg = PosAndNeg + 1
    #mix and positive transition
        if (valence == 3 and preValence == 2) or (valence == 2 and preValence == 3) :
            MixAndPos = MixAndPos + 1
    #mix and negative transition
        if (valence == 3 and preValence == 1) or (valence == 1 and preValence == 3) :
            MixAndNeg = MixAndNeg + 1
    #mix and neutral transition
        if (valence == 3 and preValence == 4) or (valence == 4 and preValence == 3) :
            MixAndNeu = MixAndNeu + 1
    #neutral and postive transition
        if (valence == 4 and preValence == 2) or (valence == 2 and preValence == 4) :
            NeuAndPos = NeuAndPos + 1
    #neutral and negative transition 
        if (valence == 4 and preValence == 1) or (valence == 1 and preValence == 4) :
            NeuAndNeg = NeuAndNeg + 1
    #Empty and positive transition
        if (valence == 0 and preValence == 2) or (valence == 2 and preValence == 0) :
            EmptyAndPos = EmptyAndPos + 1
    
    #Empty and negative transition
        if (valence == 0 and preValence == 1) or (valence == 1 and preValence == 0) :
            EmptyAndNeg = EmptyAndNeg + 1
    
    #Empty and mix transition
        if (valence == 0 and preValence == 3) or (valence == 3 and preValence == 0) :
            EmptyAndMix = EmptyAndMix + 1
    
    #Empty and neutral transition
        if (valence == 4 and preValence == 0) or (valence == -1 and preValence == 4) :
            EmptyAndNeu = EmptyAndNeu + 1
            
            
        preValence = valence
    return [emptyTran, negaTran, posiTran, mixTran, neuTran, PosAndNeg, MixAndPos, MixAndNeg, MixAndNeu, NeuAndPos, NeuAndNeg, EmptyAndPos, EmptyAndNeg, EmptyAndMix, EmptyAndNeu]


all the users should start with 60 days, if not, that's because their posts contain foreign lanaguge and they were cleaned from the data set, in this case, the valence vector starts with the first post


In [42]:
def getValenceVector(df):
    valenceVec = {}
    valences =[]
    preUser = None
    preDay = None
    preValence = []

    for valence, day, user in zip(df['negative_ny'], df['time_diff'], df['userid']):
        if preUser is None:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
                for num in range(0, addDays):
                    valences.append(0)
                valenceVec[user] = valences
            else:
                valences = [valence]
                valenceVec[user] = valences

        elif user == preUser and preDay == day+1:
            valences.append(valence)
            
        elif user == preUser and preDay != day+1:
            valences.append(valence)
            addDays = preDay - (day+1)
            for num in range(0, addDays):
                valences.append(0)
                
        elif user != preUser:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
               # print(user,addDays)
                valences = [valence]
                for num in range(0, addDays):
                    valences.insert(0, 0)
                valenceVec[user] = valences
                #print(valences)
            else:
                valences = [valence]
                valenceVec[user] = valences
                #print(valences)
                
            if preDay > 1:
                addDays = preDay
                for num in range(0, addDays):
                    preValence.append(0)
                    
                         
            
        preUser = user
        preDay = day
        preValence = valences
#     valenceVec[preUser] = valences
    return valenceVec

valenceVec = getValenceVector(time)

In [38]:
i = 0
for item in valenceVec:
    print(item, valenceVec[item])
    i = i+1
print(i)


fbd681e79ebca2cd9509218403f23ec6 [0, 2, 0, 2, 2, 0, 2, 4, 2, 4, 2, 2, 4, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 1, 4, 2, 0, 2, 1, 0, 0, 0, 5, 0, 0, 1, 1, 0, 2, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0]
f8708568eff1535c85247b28dc126f9b [2, 2, 2, 1, 1, 4, 2, 4, 2, 1, 2, 2, 1, 1, 4, 2, 2, 2, 2, 1, 1, 1, 4, 4, 2, 1, 3, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 1, 4, 2, 1, 4, 4, 1, 4, 2, 1, 1, 4, 2, 2, 2, 1, 4, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 4, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 4, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 4, 1, 4, 0, 2, 3, 2, 2, 4, 4, 4, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 4, 2, 2, 1, 2, 2, 2, 1, 4, 2, 2, 2, 2, 1, 2, 2]
f49956142aab2ad74decfd49a103c2f1 [0, 4, 1, 1, 1, 1, 4, 4, 0, 1, 4, 1, 0, 0, 0, 0, 0, 2, 2, 4, 2, 4, 0, 0, 0, 0, 4, 1, 1, 1, 4, 0, 2, 0, 0, 0, 2, 1, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 4, 0, 0, 2, 2, 2, 2, 1, 2, 1, 4, 4, 1, 2, 4, 2, 2

In [91]:
count = 0
negativeD = []
positiveD = []
neutralD = []
mixed = []
empty = []
useridL = []
for userid in valenceVec:
    negativeD.append(valenceVec[userid].count(1))
    positiveD.append(valenceVec[userid].count(2))
    neutralD.append(valenceVec[userid].count(4))
    mixed.append(valenceVec[userid].count(3))
    empty.append(valenceVec[userid].count(0))
    useridL.append(userid)
    print('user {} has  {} empty days, {}, negative posts, {} positive posts {}, neutral posts and mix posts {}'.format(userid, valenceVec[userid].count(0), valenceVec[userid].count(1), valenceVec[userid].count(2), valenceVec[userid].count(4), valenceVec[userid].count(3)))
#     count += 1
#     if count == 5:
#         break


user fbd681e79ebca2cd9509218403f23ec6 has  31 empty days, 11, negative posts, 23 positive posts 4, neutral posts and mix posts 0
user f8708568eff1535c85247b28dc126f9b has  1 empty days, 49, negative posts, 87 positive posts 22, neutral posts and mix posts 3
user f49956142aab2ad74decfd49a103c2f1 has  30 empty days, 15, negative posts, 21 positive posts 13, neutral posts and mix posts 0
user f358b75c71d85c91ec0452c6114e3b15 has  29 empty days, 9, negative posts, 25 positive posts 3, neutral posts and mix posts 3
user ec3dc451b8a811f05ace158a0d76e32e has  43 empty days, 14, negative posts, 2 positive posts 8, neutral posts and mix posts 0
user eaeabcfe25866f0455d349bae2450d88 has  27 empty days, 11, negative posts, 28 positive posts 6, neutral posts and mix posts 1
user e9970fe8848e9505b52d414d81611de8 has  40 empty days, 5, negative posts, 17 positive posts 6, neutral posts and mix posts 2
user e076953cd3fcf8927b8f21574c0890f3 has  28 empty days, 10, negative posts, 19 positive posts 10,

In [68]:
def getUserTransitions(valencVec):
    result = {}
    for item in valencVec:
        result[item] = getTransitions(valencVec[item])
#         print(result)
    return result
        
TransitionStates = getUserTransitions(valenceVec)   


create table for statistics

In [97]:
df = pd.DataFrame(np.array(negativeD).reshape(74,1), columns=['NegativePosts'])
df['PositivePosts'] = positiveD
df['NeutralPosts'] = neutralD
df['MixedPosts'] = mixed
df['EmptyPosts'] = empty
df['userid'] = useridL
df[0:5]

,NegativePosts,PositivePosts,NeutralPosts,MixedPosts,EmptyPosts,userid
0,11,23,4,0,31,fbd681e79ebca2cd9509218403f23ec6
1,49,87,22,3,1,f8708568eff1535c85247b28dc126f9b
2,15,21,13,0,30,f49956142aab2ad74decfd49a103c2f1
3,9,25,3,3,29,f358b75c71d85c91ec0452c6114e3b15
4,14,2,8,0,43,ec3dc451b8a811f05ace158a0d76e32e


In [64]:
df.describe()

,NegativePosts,PositivePosts,NeutralPosts,MixedPosts,EmptyPosts
count,74.000000,74.00000,74.000000,74.000000,74.000000
mean,16.337838,27.00000,10.243243,1.189189,28.783784
std,15.393716,21.62254,8.098386,1.677210,11.603342
min,0.000000,2.00000,0.000000,0.000000,0.000000
25%,7.250000,17.00000,5.000000,0.000000,23.250000
50%,12.000000,20.50000,8.000000,1.000000,30.000000
75%,19.750000,26.75000,13.750000,2.000000,37.000000
max,107.000000,100.00000,39.000000,11.000000,44.000000


In [65]:
#pickle
import pickle
with open(path + 'newScripts/moodVector/moodVectorsData/MoodVecDes3Empty.pickle', 'wb') as handle:
    pickle.dump(valenceVec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [66]:
#write result as csv
#header = ['userid','vector']
with open(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTransitions(empty).csv','w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(i for i in header)
    writer.writerow(TransitionStates.keys())
    for row in zip(*TransitionStates.values()):
        writer.writerow(list(row))

In [80]:
#read csv file 
file = pd.read_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTransitions(empty).csv')
file = file.transpose()
file.columns = ['emptyTran', 'negaTran', 'posiTran', 'mixTran', 'neuTran', 'PosAndNeg', 'MixAndPos', 'MixAndNeg', 'MixAndNeu', 'NeuAndPos', 'NeuAndNeg', 'EmptyAndPos', 'EmptyAndNeg', 'EmptyAndMix', 'EmptyAndNeu']
file.shape

(74, 15)

In [82]:
#add one column as row sum
file['allPosts'] = file[1:14].sum(axis=1) 
file[1:5]

,emptyTran,negaTran,posiTran,mixTran,neuTran,PosAndNeg,MixAndPos,MixAndNeg,MixAndNeu,NeuAndPos,NeuAndNeg,EmptyAndPos,EmptyAndNeg,EmptyAndMix,EmptyAndNeu,allPosts
f8708568eff1535c85247b28dc126f9b,0,22,56,0,4,36,4,2,0,19,16,2,0,0,0,161.0
f49956142aab2ad74decfd49a103c2f1,19,5,9,0,2,8,0,0,0,5,9,11,3,0,4,150.0
f358b75c71d85c91ec0452c6114e3b15,11,3,8,1,0,5,1,0,0,4,0,24,7,3,2,138.0
ec3dc451b8a811f05ace158a0d76e32e,36,6,0,0,2,2,0,0,0,2,4,0,9,0,2,126.0


compute transition states probability,  we compute the pobability by dividing the transition with number of all posts

In [83]:
Tranprob = file.apply(lambda x: x/file.iloc[:,-1])
Tranprob['userid'] = Tranprob.index
Tranprob[1:5]

,emptyTran,negaTran,posiTran,mixTran,neuTran,PosAndNeg,MixAndPos,MixAndNeg,MixAndNeu,NeuAndPos,NeuAndNeg,EmptyAndPos,EmptyAndNeg,EmptyAndMix,EmptyAndNeu,allPosts,userid
f8708568eff1535c85247b28dc126f9b,0.000000,0.136646,0.347826,0.000000,0.024845,0.223602,0.024845,0.012422,0.0,0.118012,0.099379,0.012422,0.000000,0.000000,0.000000,1.0,f8708568eff1535c85247b28dc126f9b
f49956142aab2ad74decfd49a103c2f1,0.126667,0.033333,0.060000,0.000000,0.013333,0.053333,0.000000,0.000000,0.0,0.033333,0.060000,0.073333,0.020000,0.000000,0.026667,1.0,f49956142aab2ad74decfd49a103c2f1
f358b75c71d85c91ec0452c6114e3b15,0.079710,0.021739,0.057971,0.007246,0.000000,0.036232,0.007246,0.000000,0.0,0.028986,0.000000,0.173913,0.050725,0.021739,0.014493,1.0,f358b75c71d85c91ec0452c6114e3b15
ec3dc451b8a811f05ace158a0d76e32e,0.285714,0.047619,0.000000,0.000000,0.015873,0.015873,0.000000,0.000000,0.0,0.015873,0.031746,0.000000,0.071429,0.000000,0.015873,1.0,ec3dc451b8a811f05ace158a0d76e32e


In [84]:
Tranprob.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTransitions(Empty).csv')

correlation with all scales

In [85]:
allData = pd.read_csv( path + '/data/important_data/user_scale_post_time2.csv')
Var = allData[['userid','ope','con','ext','agr','neu','swl','CESD_sum']]
Var[1:5]

,userid,ope,con,ext,agr,neu,swl,CESD_sum
1,06165dddbf8856657d57d27600c4944c,4.75,3.00,2.74,2.45,3.90,4.0,19
2,07244b3fd4536a8eea069ab02057e220,4.85,3.30,2.58,4.30,2.30,5.8,45
3,0724fe854bd455061ba84efecdeff469,4.80,3.55,3.00,2.95,3.15,6.4,6
4,08074f06d6882e73b4db242b38be237a,3.85,2.90,2.90,2.60,3.26,3.2,12


In [86]:
compare = pd.merge(Tranprob, Var, on ='userid', how = 'left')
corMatrix = compare.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTrancorrelationMatrix(Empty).csv')
corMatrix

,emptyTran,negaTran,posiTran,mixTran,neuTran,PosAndNeg,MixAndPos,MixAndNeg,MixAndNeu,NeuAndPos,...,EmptyAndMix,EmptyAndNeu,allPosts,ope,con,ext,agr,neu,swl,CESD_sum
emptyTran,1.000000,-0.502306,-0.584246,-0.173801,-0.126661,-0.714141,-0.564598,-0.507909,-0.111101,-0.647251,...,0.081354,0.263101,NaN,0.350495,0.096835,-0.145705,-0.022970,0.151580,-0.157550,0.144473
negaTran,-0.502306,1.000000,0.809084,-0.064352,0.406433,0.821022,0.574544,0.728439,0.029155,0.592952,...,-0.197774,-0.371663,NaN,-0.242078,-0.040684,0.237565,-0.270372,-0.349995,-0.092345,-0.055615
posiTran,-0.584246,0.809084,1.000000,-0.020744,0.241188,0.945001,0.785281,0.681652,-0.121301,0.657511,...,-0.173179,-0.557847,NaN,-0.476023,0.073035,0.056506,-0.032268,-0.497919,0.138801,-0.193693
mixTran,-0.173801,-0.064352,-0.020744,1.000000,-0.307196,-0.132548,0.079963,-0.183735,-0.144163,-0.191512,...,0.563528,-0.073043,NaN,-0.179896,0.333633,-0.088125,0.591427,-0.053063,0.441401,0.033811
neuTran,-0.126661,0.406433,0.241188,-0.307196,1.000000,0.194267,0.174602,0.400639,0.593721,0.701420,...,-0.411294,0.313586,NaN,-0.252491,-0.021786,0.203075,-0.501773,0.189409,0.275566,-0.260684
PosAndNeg,-0.714141,0.821022,0.945001,-0.132548,0.194267,1.000000,0.803173,0.667643,-0.129665,0.681494,...,-0.275968,-0.553421,NaN,-0.411756,-0.118222,0.050652,-0.078917,-0.423215,-0.033925,-0.104587
MixAndPos,-0.564598,0.574544,0.785281,0.079963,0.174602,0.803173,1.000000,0.397419,-0.139944,0.560589,...,0.015467,-0.677011,NaN,-0.253783,-0.158889,-0.027017,0.251278,-0.463967,0.015519,0.029103
MixAndNeg,-0.507909,0.728439,0.681652,-0.183735,0.400639,0.667643,0.397419,1.000000,0.523930,0.614877,...,-0.087439,-0.304561,NaN,-0.045699,-0.094763,0.501689,-0.452894,-0.098142,-0.091205,0.106656
MixAndNeu,-0.111101,0.029155,-0.121301,-0.144163,0.593721,-0.129665,-0.139944,0.523930,1.000000,0.326161,...,-0.008358,0.378912,NaN,0.147444,-0.213800,0.515363,-0.515240,0.546434,-0.020709,0.231937
NeuAndPos,-0.647251,0.592952,0.657511,-0.191512,0.701420,0.681494,0.560589,0.614877,0.326161,1.000000,...,-0.457327,-0.129507,NaN,-0.432273,-0.107072,0.092809,-0.325564,-0.069120,0.391995,-0.317811


In [101]:
compareFreq = pd.merge(df, Var, on ='userid', how = 'left')
corMatrix = compareFreq.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceFreqMatrix(Empty).csv')
corMatrix

,NegativePosts,PositivePosts,NeutralPosts,MixedPosts,EmptyPosts,ope,con,ext,agr,neu,swl,CESD_sum
NegativePosts,1.000000,0.575723,0.545127,0.496761,-0.739357,-0.137392,-0.008357,-0.159222,-0.271421,0.193769,-0.140572,0.157643
PositivePosts,0.575723,1.000000,0.394826,0.575285,-0.856554,-0.160556,-0.058299,0.096967,-0.006243,0.151840,-0.027382,0.097885
NeutralPosts,0.545127,0.394826,1.000000,0.259793,-0.642756,-0.092921,-0.173541,-0.178013,-0.223745,0.046922,-0.098969,0.036401
MixedPosts,0.496761,0.575285,0.259793,1.000000,-0.520158,-0.032658,0.045069,-0.074525,0.103119,0.150321,-0.088840,0.127829
EmptyPosts,-0.739357,-0.856554,-0.642756,-0.520158,1.000000,0.170410,0.009196,0.032385,0.144273,-0.167209,0.045514,-0.106228
ope,-0.137392,-0.160556,-0.092921,-0.032658,0.170410,1.000000,-0.075814,0.305311,0.017626,-0.209973,-0.078370,-0.227367
con,-0.008357,-0.058299,-0.173541,0.045069,0.009196,-0.075814,1.000000,0.184559,0.151372,-0.367604,0.401755,-0.402877
ext,-0.159222,0.096967,-0.178013,-0.074525,0.032385,0.305311,0.184559,1.000000,0.182808,-0.345980,0.204759,-0.351483
agr,-0.271421,-0.006243,-0.223745,0.103119,0.144273,0.017626,0.151372,0.182808,1.000000,-0.277535,0.246162,-0.193154
neu,0.193769,0.151840,0.046922,0.150321,-0.167209,-0.209973,-0.367604,-0.345980,-0.277535,1.000000,-0.402645,0.614550


Now we write the valence vector as csv file 